In [1]:
import skimage.io
import dask.array as da
import dask
from dask import delayed
from pathlib import Path
import napari
from natsort import natsorted
import numpy as np
from napari_flim_phasor_calculator._reader import get_current_tz
import zarr

Define read 2d and 3d functions

In [2]:
# folder_path = r"C:\Users\mazo260d\Desktop\Conni_BiA_PoL\embryo_FLIM_data\raw_data_embryo_stack_3tps_43pl_2ch\output_as_tif"
# folder_path = r"C:\Users\mazo260d\Desktop\Conni_BiA_PoL\stack_smaller_as_tif"
folder_path = r"C:\Users\mazo260d\Desktop\Conni_BiA_PoL\stack_as_tif"

folder_path = Path(folder_path)

Get max slice shape

In [3]:
def read_tif_data_2D(path):
    from skimage.io import imread
    image = imread(path)
    return image

# Read all slices to get slice max shape and dtype (not ideal but for now it is OK)
slice_shape_list = []
for file_path in folder_path.iterdir():
    if file_path.suffix == '.tif':
        image_2D = read_tif_data_2D(file_path)
        slice_shape_list.append(image_2D.shape)
slice_max_shape = max(slice_shape_list)
print('last_slice_shape = ', image_2D.shape,'image_dtype = ', image_2D.dtype)
print('max_slice_shape = ', slice_max_shape)

last_slice_shape =  (2, 266, 512, 512) image_dtype =  uint16
max_slice_shape =  (2, 276, 512, 512)


Get max z slices and max t timepoints from filepaths

In [4]:
file_paths = [file_path for file_path in folder_path.iterdir() if file_path.suffix == '.tif']

# Get max z slices by reading all file names
def get_max_zslices(file_paths):
    max_z = max([get_current_tz(file_path) for file_path in file_paths if file_path.suffix == '.tif'])[1]
    if max_z is None:
        return 1
    return max_z
def get_max_time(file_paths):
    max_time = max([get_current_tz(file_path) for file_path in file_paths if file_path.suffix == '.tif'])[0]
    if max_time is None:
        return 1
    return max_time

max_z = get_max_zslices(file_paths)
print('max_z slices = ', max_z)
max_t = get_max_time(file_paths)
print('max_time = ', max_t)

max_z slices =  65
max_time =  1


In [5]:
stack_shape = (max_t, *slice_max_shape[:-2], max_z, *slice_max_shape[-2:])
stack_shape

(1, 2, 276, 65, 512, 512)

Get list of paths

In [6]:
t_path_list = []
z_path_list = []
file_paths = natsorted(file_paths)
previous_t = 1
for file_path in file_paths:
    if file_path.suffix == '.tif':
        current_t, current_z = get_current_tz(file_path)
        if current_t is not None:
            if current_t > previous_t:
                t_path_list.append(z_path_list)
                z_path_list = []
                previous_t = current_t
            z_path_list.append(file_path)

# If no timepoints, z+path_list is file_paths
if current_t is None:
    z_path_list = file_paths

# Append last timepoint
t_path_list.append(z_path_list)

In [7]:
url = folder_path / (folder_path.stem + '2.zarr')
# Using zarr to automatically guees chunk sizes
z1 = zarr.open(url, mode='w', shape=stack_shape, dtype=image_2D.dtype)
# Using dask to rechunk micro-time axis
dask_array = da.from_zarr(url)
print(dask_array.chunks)
dask_array = dask_array.rechunk(chunks={-4: -1})
print(dask_array.chunks)
# Overwriting previous zarr rechunked
da.to_zarr(dask_array, url, overwrite=True)
# z = zarr.zeros(stack_shape, path = url, dtype=image_2D.dtype)
# z1.shape

((1,), (1, 1), (35, 35, 35, 35, 35, 35, 35, 31), (9, 9, 9, 9, 9, 9, 9, 2), (64, 64, 64, 64, 64, 64, 64, 64), (128, 128, 128, 128))
((1,), (1, 1), (276,), (9, 9, 9, 9, 9, 9, 9, 2), (64, 64, 64, 64, 64, 64, 64, 64), (128, 128, 128, 128))


In [8]:
z1 = zarr.open(url, mode='r+')

In [9]:
z1.chunks

(1, 1, 276, 9, 64, 128)

In [10]:
for i, z_paths in enumerate(t_path_list):
    for j, path in enumerate(z_paths):
        data = read_tif_data_2D(path)
        z1[i,:data.shape[0],:data.shape[1],j, :data.shape[2], :data.shape[3]] = read_tif_data_2D(path)

In [11]:
viewer = napari.Viewer()

In [12]:
viewer.add_image(z1, channel_axis=1)

[<Image layer 'Image' at 0x2469d740c10>,
 <Image layer 'Image [1]' at 0x2469d9a6af0>]